In [1]:
# Importing the important and required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB

In [2]:
# Importing the data
Spam = pd.read_csv(r"C:\Users\MOHANRAJ\Desktop\Projects\Machine Learning\Email Spam Detection\spam.csv")
Spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# To find if nan values present
Spam.isnull().sum()

Category    0
Message     0
dtype: int64

In [7]:
# Creating a column using one hot encoding
Spam["Spam"]=Spam["Category"].map({"ham":0,"spam":1})

# Droping the Category column
Spam = Spam.drop("Category",axis=1)

Spam.head()

,Message,Spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [11]:
# Now we are going to split the data into Train and Test datasets
# Using sklearn.model --> train_test_split

from sklearn.model_selection import train_test_split
Xinp_train,Xinp_test,Yout_train,Yout_test = train_test_split(Spam.Message,Spam.Spam)



# Message are in categorical needed to change into numerical using countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
CV=CountVectorizer()
CV.fit(Xinp_train)
xinp_train=CV.transform(Xinp_train)
xinp_test=CV.transform(Xinp_test)

xinp_train

<4179x7468 sparse matrix of type '<class 'numpy.int64'>'
	with 55630 stored elements in Compressed Sparse Row format>

In [14]:
# Model building
Model=MultinomialNB()
Model.fit(xinp_train,Yout_train)

MultinomialNB()

In [15]:
# To Check the score of the Test datasets
Model.score(xinp_test,Yout_test)

0.9849246231155779

In [18]:
# For Example checking the model
emails = ['I linked the documents for yor reference',
          'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!']
email=CV.transform(emails)

# Prediction
Model.predict(email)

array([0, 1], dtype=int64)

In [22]:
# Save the model
import pickle
with open("multinominal.pkl","wb") as f:
    pickle.dump(model,f)
    
# Using pipelines
from sklearn.pipeline import Pipeline
clf = Pipeline([('vectorizer', CountVectorizer()),
                ('nb', MultinomialNB())])

In [63]:
# Fitting the Train dataset
clf.fit(Xinp_train,Yout_train)

# Prediction
clf.predict(emails)


array([0, 1], dtype=int64)

In [71]:
# Creating a model which Accepts user input and determine wether it is Spam or not
E_mail_input=list(input().split("|"))

# Applying it in the Model created above
status = clf.predict(E_mail_input)

# Creating the dataFrame with the above data
Spam_Detection = pd.DataFrame(E_mail_input)
Spam_Detection["spam"] = status.tolist()
Spam_Detection.columns = ["E-mail","spam"]

# Creating a column using one hot encoding
Spam_Detection["Status"] = Spam_Detection["spam"].map({0:"Not Spam",1:"Spam"})

# Droping the spam column
Spam_Detection = Spam_Detection.drop("spam",axis=1)

Spam_Detection

Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward! | Hey mohan, can we get together to watch footbal game tomorrow?


,E-mail,Status
0,"Upto 20% discount on parking, exclusive offer ...",Spam
1,"Hey mohan, can we get together to watch footb...",Not Spam
